# Compute Mc from strain for all the sources 

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy

from rtree import index
from prettytable import PrettyTable

import scipy.constants as constants  
import scipy.special as special

from oqmbt.oqt_project import OQtProject
from oqmbt.tools.strain import get_moment_for_polygon
from oqmbt.tools.mfd import get_moment_from_mfd

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]

In [ ]:
p = PrettyTable(["ID", "Mc", "Mx", "Mcs"])
p.align["ID"] = 'l'
p.align["Mc"] = 'r'
p.align["Mx"] = 'r'
p.align["Mcs"] = 'r'

## Computing scalar moment from strain 

In [ ]:
shear_modulus = model.shear_modulus 
mo_dict = {}

for key in model.sources:
    source = model.sources[key]
    
    if re.match('AreaSource', source.source_type) and 'a_gr' in source.__dict__:
        #
        # get parameters
        mo_tot = source.mo_strain
        a_gr = source.a_gr
        b_gr = source.b_gr
        #
        #
        threshold_mag = 6.0
        efficiency = 1.0
        beta = b_gr*2./3. # See Fig. 3 of Kagan and Jackson (2015)
        #
        alpha = 10**(a_gr-b_gr*threshold_mag) 
        mo_low = 10.**(1.5*threshold_mag+9.0)
        #
        # truncated pareto distribution
        num = mo_tot*(1.-beta)*efficiency
        den = alpha*(mo_low**beta)*beta
        mx = (num / den)**(1./(1.-beta))
        m_mx = (numpy.log10(mx)-9.0)/1.5
        #
        # gamma distribution
        gamma = special.gamma(2.-beta) 
        den = den*gamma
        mc = (num / den)**(1./(1.-beta))
        m_mc = (numpy.log10(mc)-9.0)/1.5
        #
        # Simplified formula for the calculation of corner moment/magnitude
        # see Yufang slides and Kagan (2002) eq. 15
        den = alpha*(mo_low**beta)*gamma
        mcs = (num/den)**(1/(1-beta)) 
        m_mcs = (numpy.log10(mcs)-9.0)/1.5
        # 
        # adding a row to the table
        p.add_row([key, "%.2f" % m_mc, "%.2f" % m_mx, "%.2f" % m_mcs])
        print(key, mcs)
        #
        # updating source parameters
        source.mo_mx = mx
        source.mo_mc = mc
        source.mo_mcs = mcs
        #
        # updating source
        model.sources[key] = source
        
    else:
        if re.match('AreaSource', source.source_type):
            print('Skipping area source ', key)
        
print(p)
oqtkp.models[model_id] = model
oqtkp.save()